In [1]:
## 1. install and Load Libraries
!pip install opencv-python
!pip install tensorflow


from google.colab import drive
drive.mount('C:/Users/subas/Downloads/downloads1/Data/data')

from sklearn import metrics
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras import Sequential, Input, Model
from tensorflow.keras import layers
from tensorflow.keras import losses
from tensorflow.keras.callbacks import EarlyStopping


ModuleNotFoundError: No module named 'google.colab'

In [ ]:
#Load and Preprocess the Data: First, you'll need to load your images from both datasets. Preprocessing typically involves resizing all images to a uniform size (as required by the input of your chosen CNN model), converting them to grayscale or other color scales (if necessary), and normalizing the pixel values to be in the range 0-1.
import cv2
import numpy as np
import os

def load_images_from_folder(folder):
    images = []
    for filename in os.listdir(folder):
        img = cv2.imread(os.path.join(folder, filename))
        if img is not None:
            img = cv2.resize(img, (28, 28))  # Resize image
            img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)  # Convert to grayscale
            img = img / 255.0  # Normalize
            images.append(img)
    return images

vehicle_images = load_images_from_folder('C:/Users/Downloads/downloads1/Data/data/vehicles')

non_vehicle_images = load_images_from_folder('C:/Users/downloads/downloads1/Data/data/non-vehicles')

# Combine both images 
X = vehicle_images + non_vehicle_images

# create labels (stored as list)
#  1 means vehicle
#  0 means non-vehicle

y = [1] * len(vehicle_images) + [0] * len(non_vehicle_images)


In [ ]:
from sklearn.model_selection import train_test_split


# Convert to numpy arrays
X = np.array(X)
y = np.array(y)

print(X.shape)
print(y.shape)

# Split into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

print(X_train.shape)
print(X_test.shape)
print(y_train.shape)
print(y_test.shape)


In [ ]:
# Add 2-pixel padding to the height and width dimensions of the x_training data and normalize the pixel values to be between 0 and 1
X_train = tf.pad(X_train, [[0,0], [2,2], [2,2]])/255 

# Expand the dimensions of x_train to add an extra channel dimension
X_train = tf.expand_dims(X_train, axis=3, name=None)

# Repeat x_train 3 times along the channel axis to mimic RGB channels
X_train = tf.repeat(X_train, 3, axis=3)

# Perform the same operations on the testing data
X_test = tf.pad(X_test, [[0,0], [2,2], [2,2]])/255
X_test = tf.expand_dims(X_test, axis=3, name=None)
X_test = tf.repeat(X_test, 3, axis=3)

# Split the last 2000 examples from the x_training data to be used as validation data
X_val = X_train[-2000:, :, :, :]
y_val = y_train[-2000:]

# The remaining data is kept for x_training
X_train = X_train[:-2000, :, :, :]
y_train = y_train[:-2000]

# Print the shapes of the datasets to check everything went as expected
print(X_train.shape, X_val.shape, X_test.shape)
print(y_train.shape, y_val.shape, y_test.shape)


In [ ]:
# 3. Loading Pretrained VGG Model
base_model = tf.keras.applications.VGG16(weights = 'imagenet',
                                         include_top=False,
                                         input_shape = (32,32,3))
for layer in base_model.layers:
    layer.trainable = False

# Corrected indentation
x = layers.Flatten()(base_model.output)
x = layers.Dense(1000, activation='relu')(x)
predictions = layers.Dense(10, activation = 'softmax')(x)


In [ ]:
# 4. Compile and Train Model
head_model = Model(inputs = base_model.input, outputs = predictions)

head_model.compile(optimizer = 'adam', loss = losses.sparse_categorical_crossentropy, metrics = ['accuracy'])

history = head_model.fit(X_train, y_train, batch_size=64, epochs=10, validation_data = (X_val, y_val))


In [ ]:
# 5. Plot Learning Curve
fig, axs = plt.subplots(2, 1, figsize=(6,6))

axs[0].plot(history.history['loss'])
axs[0].plot(history.history['val_loss'])
axs[0].title.set_text('Training Loss vs Validation Loss')
axs[0].set_xlabel('Epochs')
axs[0].set_ylabel('Loss')
axs[0].legend(['Train', 'Val'])

axs[1].plot(history.history['accuracy'])
axs[1].plot(history.history['val_accuracy'])
axs[1].title.set_text('Training Accuracy vs Validation Accuracy')
axs[1].set_xlabel('Epochs')
axs[1].set_ylabel('Accuracy')
axs[1].legend(['Train', 'Val'])

plt.show()


In [ ]:
# 6. Evaluate Test Accuracy

score = head_model.evaluate(X_test, y_test, verbose=0)

print("\n", "Test Accuracy:", score[1])


In [ ]:
# 7. Finetune (unfreeze):

## unfreezing
base_model = tf.keras.applications.VGG16(weights = 'imagenet',
                                         include_top=False,
                                         input_shape = (32,32,3))
for layer in base_model.layers:
    layer.trainable = True

In [ ]:
print("Number of layers in the base model:", len(base_model.layers))

In [ ]:
fine_tune_at = 10

for layer in base_model.layers[:fine_tune_at]:
    layer.trainable = False

x = layers.Flatten()(base_model.output)
x = layers.Dense(1000, activation='relu')(x)
predictions = layers.Dense(10, activation = 'softmax')(x)

In [ ]:
base_model.summary()

In [ ]:
len(base_model.trainable_variables)

In [ ]:
#  8. Compile and Train the Model

head_model = Model(inputs = base_model.input, outputs = predictions)

head_model.compile(optimizer = 'adam', loss = losses.sparse_categorical_crossentropy, metrics = ['accuracy'])


In [ ]:
history_fine = head_model.fit(X_train, y_train, batch_size=64, epochs=10, validation_data = (X_val, y_val))

In [ ]:
# 9. Plot Learning Curve
fig, axs = plt.subplots(2, 1, figsize=(6,6))

axs[0].plot(history_fine.history['loss'])
axs[0].plot(history_fine.history['val_loss'])
axs[0].title.set_text('Training Loss vs Validation Loss')
axs[0].set_xlabel('Epochs')
axs[0].set_ylabel('Loss')
axs[0].legend(['Train', 'Val'])

axs[1].plot(history_fine.history['accuracy'])
axs[1].plot(history_fine.history['val_accuracy'])
axs[1].title.set_text('Training Accuracy vs Validation Accuracy')
axs[1].set_xlabel('Epochs')
axs[1].set_ylabel('Accuracy')
axs[1].legend(['Train', 'Val'])

plt.show()



In [ ]:
# 10. Evaluate the Test Accuracy

test_loss, test_acc = head_model.evaluate(X_test, y_test, verbose=2)

print("\n", "Test Accuracy:", test_acc)


In [ ]:
# 11. Make prediction on the test dataset

probability_model = tf.keras.Sequential([head_model, tf.keras.layers.Softmax()])
predictions = probability_model.predict(X_test)


In [ ]:
class_names = ['Vehicle', 'Non-vehicle']

In [ ]:
def plot_image(i, predictions_array, true_label, img):
    true_label, img = true_label[i], img[i]
    plt.grid(False)
    plt.xticks([])
    plt.yticks([])
    plt.imshow(img, cmap=plt.cm.binary)

    predicted_label = np.argmax(predictions_array)
    if predicted_label == true_label:
        color='blue'
    else:
        color='red'
    
    plt.xlabel('{} {:2.0f}% ({})'.format(class_names[predicted_label], 100*np.max(predictions_array), 
                                         class_names[true_label], color=color))
    
def plot_value_array(i, predictions_array, true_label):
    true_label = true_label[i]
    plt.grid(range(10))
    plt.xticks([])
    plt.yticks([])
    thisplot = plt.bar(range(10), predictions_array, color = '#777777')
    plt.ylim([0,1])
    predicted_label = np.argmax(predictions_array)
    thisplot[predicted_label].set_color('red')
    thisplot[true_label].set_color('blue')

In [ ]:
import numpy as np
num_rows = 3
num_cols = 3
num_images = num_rows * num_cols
plt.figure(figsize=(2*2*num_cols, 2*num_rows))
for i in range(num_images):
    plt.subplot(num_rows, 2*num_cols, 2*i+1)
    plot_image(i, predictions[i], y_test, X_test)
    plt.subplot(num_rows, 2*num_cols, 2*i+2)
    plot_value_array(i, predictions[i], y_test)
plt.tight_layout()
plt.show()